# Data Modeling - Convert Excel to DuckDB Database

In [1]:
import duckdb
import numpy as np
import pandas as pd

In [2]:
file_path = './data/(US) Sample - Superstore.xlsx'

In [3]:
orders_df = pd.read_excel(file_path, sheet_name='Orders')
orders_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2021-152156,2021-11-08,2021-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2021-152156,2021-11-08,2021-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2021-138688,2021-06-12,2021-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2020-108966,2020-10-11,2020-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2020-108966,2020-10-11,2020-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2019-110422,2019-01-21,2019-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180.0,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2022-121258,2022-02-26,2022-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2022-121258,2022-02-26,2022-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2022-121258,2022-02-26,2022-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


# Create Separate Tables of Related Data

In [4]:
# Customers table
customers_df = orders_df[['Customer ID', 'Customer Name', 'Segment', 'Country/Region', 'City', 'State', 'Postal Code', 'Region']].drop_duplicates()

# Orders table
orders_table_df = orders_df[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID']].drop_duplicates()

# Products table
products_df = orders_df[['Product ID', 'Category', 'Sub-Category', 'Product Name']].drop_duplicates()

# OrderDetails table
order_details_df = orders_df[['Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit']]

In [5]:
# Create a new DuckDB database and connect to it
con = duckdb.connect('./data/superstore.duckdb')

# Register the dataframes as temporary tables in DuckDB
con.register("customers_df", customers_df)
con.register("orders_table_df", orders_table_df)
con.register("products_df", products_df)
con.register("order_details_df", order_details_df)

# Write the dataframes to the DuckDB database using SQL
con.execute("CREATE TABLE Customers AS SELECT * FROM customers_df")
con.execute("CREATE TABLE Orders AS SELECT * FROM orders_table_df")
con.execute("CREATE TABLE Products AS SELECT * FROM products_df")
con.execute("CREATE TABLE OrderDetails AS SELECT * FROM order_details_df")

## Verify Creation of Tables

In [6]:
# Verify that the tables were created successfully
print(con.execute("SHOW TABLES").fetchall())

# Optionally, you can query the tables to verify the data
print(con.execute("SELECT * FROM Customers LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM Orders LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM Products LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM OrderDetails LIMIT 5").fetchdf())

[('Customers',), ('OrderDetails',), ('Orders',), ('Products',), ('customers_df',), ('order_details_df',), ('orders_table_df',), ('products_df',)]
  Customer ID    Customer Name    Segment Country/Region             City  \
0    CG-12520      Claire Gute   Consumer  United States        Henderson   
1    DV-13045  Darrin Van Huff  Corporate  United States      Los Angeles   
2    SO-20335   Sean O'Donnell   Consumer  United States  Fort Lauderdale   
3    BH-11710  Brosina Hoffman   Consumer  United States      Los Angeles   
4    AA-10480     Andrew Allen   Consumer  United States          Concord   

            State  Postal Code Region  
0        Kentucky      42420.0  South  
1      California      90036.0   West  
2         Florida      33311.0  South  
3      California      90032.0   West  
4  North Carolina      28027.0  South  
         Order ID Order Date  Ship Date       Ship Mode Customer ID
0  CA-2021-152156 2021-11-08 2021-11-11    Second Class    CG-12520
1  CA-2021-1386

In [7]:
people_df = pd.read_excel(file_path, sheet_name='People').drop_duplicates()
returns_df = pd.read_excel(file_path, sheet_name='Returns').drop_duplicates()

In [8]:
con.register("people_df", people_df)

In [9]:
con.register("returns_df", returns_df)

con.execute("CREATE TABLE IF NOT EXISTS People AS SELECT * FROM people_df")
con.execute("CREATE TABLE IF NOT EXISTS Returns AS SELECT * FROM returns_df")

print(con.execute("SELECT * FROM People LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM Returns LIMIT 5").fetchdf())

    Regional Manager   Region
0    Sadie Pawthorne     West
1        Chuck Magee     East
2  Roxanne Rodriguez  Central
3        Fred Suzuki    South
  Returned        Order ID
0      Yes  CA-2019-100762
1      Yes  CA-2019-100867
2      Yes  CA-2019-102652
3      Yes  CA-2019-103373
4      Yes  CA-2019-103744


In [10]:
con.close()